In [2]:
import pandas as pd
import numpy as np
import sqlite3
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
EXPERIMENT_NAME = 'P3856'
RUN_NAME = 'P3856_YHE211_1_Slot1-1_1_5104'

In [4]:
MZ_TOLERANCE_PPM = 5
MZ_TOLERANCE_PERCENT = MZ_TOLERANCE_PPM * 10**-4
SCAN_TOLERANCE = 2
RT_TOLERANCE = 1

##### 3DID parameters

In [ ]:
# load the detected features
EXPERIMENT_DIR = '/data2/experiments/{}'.format(EXPERIMENT_NAME)
FEATURES_DIR = '{}/features-3did'.format(EXPERIMENT_DIR)
FEATURES_FILE = '{}/exp-{}-run-{}-features-3did.pkl'.format(FEATURES_DIR, EXPERIMENT_NAME, RUN_NAME)
FEATURES_DEDUP_FILE = '{}/exp-{}-run-{}-features-3did-dedup.pkl'.format(FEATURES_DIR, EXPERIMENT_NAME, RUN_NAME)
MZ_COLUMN_NAME = 'monoisotopic_mz'

In [26]:
# features_df = pd.read_pickle(FEATURES_FILE)

##### PASEF parameters

In [36]:
# load the detected features
EXPERIMENT_DIR = '/data2/experiments/{}'.format(EXPERIMENT_NAME)
FEATURES_DIR = '{}/recalibrated-features'.format(EXPERIMENT_DIR)
FEATURES_FILE = '{}/experiment-features.pkl'.format(FEATURES_DIR)
FEATURES_DEDUP_FILE = '{}/exp-{}-run-{}-features-pasef-dedup.pkl'.format(FEATURES_DIR, EXPERIMENT_NAME, RUN_NAME)
MZ_COLUMN_NAME = 'recalibrated_monoisotopic_mz'

In [37]:
perc_id = 29
RT_LOWER = 1650
RT_UPPER = 2200

features_df = pd.read_pickle(FEATURES_FILE)
features_df = features_df[(features_df.percolator_idx == perc_id) & (features_df.rt_apex >= RT_LOWER) & (features_df.rt_apex <= RT_UPPER)]

In [38]:
print('there are {} features prior to de-dup'.format(len(features_df)))

there are 168681 features prior to de-dup


In [39]:
# features_df['mz'] = features_df.recalibrated_monoisotopic_mz  # shorthand to reduce verbosity
features_df['mz'] = features_df[MZ_COLUMN_NAME]  # shorthand to reduce verbosity
features_df['mz_ppm_tolerance'] = features_df.mz * MZ_TOLERANCE_PERCENT / 100
features_df['mz_lower'] = features_df.mz - features_df.mz_ppm_tolerance
features_df['mz_upper'] = features_df.mz + features_df.mz_ppm_tolerance
features_df['scan_lower'] = features_df.scan_apex - SCAN_TOLERANCE
features_df['scan_upper'] = features_df.scan_apex + SCAN_TOLERANCE
features_df['rt_lower'] = features_df.rt_apex - RT_TOLERANCE
features_df['rt_upper'] = features_df.rt_apex + RT_TOLERANCE
# features_df['composite_key'] = features_df.apply(lambda row: '{},{},{}'.format(row.feature_id, row.precursor_id, row.percolator_idx), axis=1)
features_df['composite_key'] = features_df.apply(lambda row: '{},{}'.format(row.feature_id, row.precursor_id), axis=1)

In [40]:
# remove these after we're finished
columns_to_drop_l = []
columns_to_drop_l.append('mz')
columns_to_drop_l.append('mz_ppm_tolerance')
columns_to_drop_l.append('mz_lower')
columns_to_drop_l.append('mz_upper')
columns_to_drop_l.append('scan_lower')
columns_to_drop_l.append('scan_upper')
columns_to_drop_l.append('rt_lower')
columns_to_drop_l.append('rt_upper')
columns_to_drop_l.append('composite_key')


In [41]:
features_df.sort_values(by=['intensity'], ascending=False, inplace=True)

In [42]:
# see if any detections have a duplicate - if so, find the dup with the highest intensity and keep it
keep_l = []
for row in features_df.itertuples():
    dup_df = features_df[(features_df.mz > row.mz_lower) & (features_df.mz < row.mz_upper) & (features_df.scan_apex > row.scan_lower) & (features_df.scan_apex < row.scan_upper) & (features_df.rt_apex > row.rt_lower) & (features_df.rt_apex < row.rt_upper)].copy()
    # group the dups by charge - take the most intense for each charge
    for group_name,group_df in dup_df.groupby(['charge'], as_index=False):
        keep_l.append(group_df.iloc[0].composite_key)


In [43]:
# remove any features that are not in the keep list
dedup_df = features_df[features_df.composite_key.isin(keep_l)].copy()

In [44]:
number_of_dups = len(features_df)-len(dedup_df)
print('removed {} duplicates ({}% of the original detections)'.format(number_of_dups, round(number_of_dups/len(features_df)*100)))
print('there are {} detected de-duplicated features'.format(len(dedup_df)))

removed 15291 duplicates (9% of the original detections)
there are 153390 detected de-duplicated features


In [45]:
# remove the columns we added earlier
dedup_df.drop(columns_to_drop_l, axis=1, inplace=True)

In [46]:
dedup_df.sample(n=3)

,feature_id,charge,rt_apex,rt_peak_width,scan_apex,scan_peak_width,intensity,precursor_id,monoisotopic_mass,predicted_mass_error,recalibrated_monoisotopic_mass,recalibrated_monoisotopic_mz,percolator_idx
5,3785706,1,2113.78,20.31,363.23,137.23,1420.0,37857,862.988481,0.010207,862.978273,863.985549,29
3,2684204,1,1935.08,20.11,712.68,75.00,793.0,26842,700.140846,-0.001767,700.142613,701.149889,29
5,2288206,1,1878.08,20.00,354.22,99.81,783.0,22882,924.540684,0.012264,924.528420,925.535696,29


In [47]:
dedup_df.to_pickle(FEATURES_DEDUP_FILE)